In [5]:
import cvxpy as cvx
import numpy
import matplotlib.pyplot as plt

# Problem data.
n = 15
m = 10
numpy.random.seed(1)
A = numpy.random.randn(n, m)
b = numpy.random.randn(n)
# gamma must be nonnegative due to DCP rules.
gamma = cvx.Parameter(nonneg=True)

# Construct the problem.
x = cvx.Variable(m)
error = cvx.sum_squares(A*x - b)
obj = cvx.Minimize(error + gamma*cvx.norm(x, 1))
prob = cvx.Problem(obj)

# Construct a trade-off curve of ||Ax-b||^2 vs. ||x||_1
sq_penalty = []
l1_penalty = []
x_values = []
gamma_vals = numpy.logspace(-4, 6)
for val in gamma_vals:
    gamma.value = val
    prob.solve()
    # Use expr.value to get the numerical value of
    # an expression in the problem.
    sq_penalty.append(error.value)
    l1_penalty.append(cvx.norm(x, 1).value)
    x_values.append(x.value)

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.figure(figsize=(6,10))

# Plot trade-off curve.
plt.subplot(211)
plt.plot(l1_penalty, sq_penalty)
plt.xlabel(r'\|x\|_1', fontsize=16)
plt.ylabel(r'\|Ax-b\|^2', fontsize=16)
plt.title('Trade-Off Curve for LASSO', fontsize=16)

# Plot entries of x vs. gamma.
plt.subplot(212)
for i in range(m):
    plt.plot(gamma_vals, [xi[i] for xi in x_values])
plt.xlabel(r'\gamma', fontsize=16)
plt.ylabel(r'x_{i}', fontsize=16)
plt.xscale('log')
plt.title(r'\text{Entries of x vs. }\gamma', fontsize=16)

plt.tight_layout()
plt.show()

RuntimeError: LaTeX was not able to process the following string:
'\\\\|x\\\\|_1'
Here is the full report generated by LaTeX: 

This is pdfTeX, Version 3.1415926-2.5-1.40.14 (TeX Live 2013)
 restricted \write18 enabled.
entering extended mode
(./d28b65ed2dc14bb8bc2b1cf801c9532b.tex
LaTeX2e <2011/06/27>
Babel <3.9f> and hyphenation patterns for 78 languages loaded.
(/usr/local/texlive/2013/texmf-dist/tex/latex/base/article.cls
Document Class: article 2007/10/19 v1.4h Standard LaTeX document class
(/usr/local/texlive/2013/texmf-dist/tex/latex/base/size10.clo))
(/usr/local/texlive/2013/texmf-dist/tex/latex/type1cm/type1cm.sty)
(/usr/local/texlive/2013/texmf-dist/tex/latex/base/textcomp.sty
(/usr/local/texlive/2013/texmf-dist/tex/latex/base/ts1enc.def))
(/usr/local/texlive/2013/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/local/texlive/2013/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/local/texlive/2013/texmf-dist/tex/generic/oberdiek/ifpdf.sty)
(/usr/local/texlive/2013/texmf-dist/t

We compute a trade-off curve for a Lasso problem using basic serial computation, parallel processing, and warm-start.

In [ ]:
from cvxpy import *
import numpy as np
import cvxopt
from multiprocessing import Pool
import time

# Problem data.
n = 1000
m = 500
np.random.seed(1)
A = np.random.randn(n,m)
b = np.random.randn(n)
gamma = Parameter(nonneg=True)

# Construct the problem.
x = Variable(m)
objective = Minimize(sum_squares(A*x - b) + gamma*norm(x, 1))
p = Problem(objective)

# Assign a value to gamma and find the optimal x.
def get_x(gamma_value, warm_start=False):
    gamma.value = gamma_value
    result = p.solve()
    return x.value

gamma_max = norm(A.T.dot(b), 'inf').value
gammas = gamma_max*np.logspace(-4, 0, num=100)[::-1]

In [ ]:
#Serial computation.
start = time.time()
x_values = [get_x(value) for value in gammas]
print "Time", time.time() - start

# Parallel computation.
pool = Pool(processes = 4)
start = time.time()
par_x = pool.map(get_x, gammas)
print "Time", time.time() - start

pool = Pool(processes = 32)
start = time.time()
par_x = pool.map(get_x, gammas)
print "Time", time.time() - start
# print "hello"
# for v1,v2 in zip(x_values, par_x):
#     if np.linalg.norm(v1 - v2) > 1e-5:
#         print "error"
